### Generate stimulus:

In [83]:
import numpy as np
from scipy.signal import convolve2d

actBits = 3
weightBits = 3

nActsX = 6
nActsY = 6
nWeights = 3
nOutsX = nActsX - nWeights + 1

a = np.random.randint(-2**(actBits-1),2**(actBits-1)-1,size=(nActsX,nActsY))
w = np.random.randint(-2**(weightBits-1),2**(weightBits-1)-1,size=(nWeights,nWeights))
o = convolve2d(a,w[::-1].T[::-1].T,mode='valid')

print(f'PE cluster must be {nOutsX} PEs wide and {nWeights} PEs high ')

PE cluster must be 4 PEs wide and 3 PEs high 


In [84]:
a

array([[ 1,  0,  1, -2, -2,  1],
       [ 2, -4,  2, -1, -4, -4],
       [-1,  0,  2,  2,  1, -3],
       [-1, -3,  0, -1,  1, -3],
       [ 0, -4,  0,  2, -3, -1],
       [ 2,  0, -3, -3, -1, -1]])

In [85]:
w

array([[ 2, -2,  2],
       [-4, -2, -1],
       [-2,  0, -1]])

In [86]:
o

array([[  2,   5,  -5,  17],
       [ 20, -13, -16,  -4],
       [ 12,  19,   6,  -2],
       [ 11,   9,  10,  -4]])

### Prepare scan chains for multicast ID

In [87]:
ids_acts = np.zeros((nOutsX+1,nWeights))
for x,idx in enumerate(ids_acts):
    for y,id in enumerate(idx):
        ids_acts[x][y] = x + y
ids_acts[-1] = [0,0,0] # Only zeros for the Y-bus for now (you only need it for multichannel convolutions)
ids_acts

array([[0., 1., 2.],
       [1., 2., 3.],
       [2., 3., 4.],
       [3., 4., 5.],
       [0., 0., 0.]])

In [88]:
ids_weights = np.zeros((nOutsX+1,nWeights))
for x,idx in enumerate(ids_weights):
    for y,id in enumerate(idx):
        ids_weights[x][y] = x
ids_weights[-1] = [0,0,0] # Only zeros for the Y-bus for now (you only need it for multichannel convolutions)
ids_weights

array([[0., 0., 0.],
       [1., 1., 1.],
       [2., 2., 2.],
       [3., 3., 3.],
       [0., 0., 0.]])

### Save stimulus:

In [89]:
path = '/home/lquizon/lawrence-workspace/eyeriss/tb/cluster'
np.savetxt(path + '/a.txt',a,'%i')
np.savetxt(path + '/w.txt',w,'%i')
np.savetxt(path + '/o.txt',o,'%i')
np.savetxt(path + '/ids_acts.txt',ids_acts,'%i')
np.savetxt(path + '/ids_weights.txt',ids_weights,'%i')

### Have a look at existing stimulus:

In [90]:
import numpy as np
path = '.'
a = np.loadtxt(path + '/a.txt')
w = np.loadtxt(path + '/w.txt')
o= np.loadtxt(path+'/o.txt')  

### Row-stationary Forensics

In [91]:
osize = nActsX-nWeights+1
sequence_array = [[
    (0, 0), # 0,0
    (1, 1), # 
    (2, 2),
    ],[
    (0, 1),
    (1, 2),
    (2, 3),
    ],[
    (0, 2),
    (1, 3),
    (2, 4)
    ]
]
ps = []
psr = []

for sequence in sequence_array:
    psr = []
    for wrow,arow in sequence:
        psr.append(np.convolve(a[arow],w[wrow][::-1],'valid'))   
    ps.append(psr)

np.array(ps)

# Expected PE partial sums:

array([[[  4.,  -6.,   2.,   2.],
        [ -2.,  13.,  -2.,  16.],
        [  0.,  -2.,  -5.,  -1.]],

       [[ 16., -14.,  -2.,  -2.],
        [  2.,  -6., -13.,  -7.],
        [  2.,   7.,  -1.,   5.]],

       [[  2.,   0.,   2.,  -4.],
        [ 10.,  13.,   1.,   5.],
        [  0.,   6.,   3.,  -3.]]])

In [92]:
ress = []
for psr in ps:
    res = psr.copy()
    for i in range(1,len(res)):
        res[i] += res[i-1]
    ress.append(res)
np.array(ress)

# Expected partial sum outputs in systolic passing:

array([[[  4.,  -6.,   2.,   2.],
        [  2.,   7.,   0.,  18.],
        [  2.,   5.,  -5.,  17.]],

       [[ 16., -14.,  -2.,  -2.],
        [ 18., -20., -15.,  -9.],
        [ 20., -13., -16.,  -4.]],

       [[  2.,   0.,   2.,  -4.],
        [ 12.,  13.,   3.,   1.],
        [ 12.,  19.,   6.,  -2.]]])